In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import itertools
import pickle

from pix_framework.statistics.distribution import DurationDistribution, get_best_fitting_distribution, DistributionType

import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

In [2]:
file_path = '../../../data/BPI Challenge 2017.xes'
event_log = pm4py.read_xes(file_path)

/home/michelkunkler/.local/share/virtualenvs/TaskExecutionTimeMining--mAzMuWa/lib/python3.10/site-packages/pm4py/util/dt_parsing/parser.py:77: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [ ]:
from dumas_model import DumasModel

dm = DumasModel(event_log)


with open('../../../models/bpic_2017_dumas_model/dumas_model.pickle', 'wb') as handle:
    pickle.dump(dm, handle)


/home/michelkunkler/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log[timestamp_name + complete_name_gen] - start_end_event_log[timestamp_name + start_name_gen]
/home/michelkunkler/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [ ]:
from pm4py.objects.petri_net import semantics
from scipy.stats import gaussian_kde


get_real_end_time = lambda case_name : event_log[event_log['case:concept:name'] == case_name]['time:timestamp'].max().timestamp()

def sample_end_time(case_name):
    example_application = event_log[event_log['case:concept:name'] == case_name].copy()
    example_application['new_concept'] = example_application['concept:name'] +  '__' + example_application['EventID'] + '__' + example_application['lifecycle:transition']
    example_application = example_application[example_application['concept:name'].str.startswith('W_')]

    net, im, fm = pm4py.discover_petri_net_inductive(example_application, activity_key='new_concept', case_id_key='case:concept:name', timestamp_key='time:timestamp')
    get_enabled_tasks = lambda marking : list(semantics.enabled_transitions(net, marking))

    marking = im

    current_time = example_application['time:timestamp'].min().timestamp()
    start_time = current_time
    
    finish_time = dict() 
    activity_count = collections.defaultdict(lambda : collections.defaultdict(int))
    resource_count = collections.defaultdict(lambda : collections.defaultdict(int))
    while get_enabled_tasks(marking):
        pn_task = get_enabled_tasks(marking)[0]
        task, workitem, action = pn_task.label.split('__')
        row = example_application[example_application['new_concept'] == pn_task.label].iloc[0]
        
        resource = row['org:resource']
        concept_name = row['concept:name']
        activity_count[action][row['concept:name']] += 1
        resource_count[action][row['org:resource']] += 1
    
        #print(action, concept_name, resource)
        
        if action == 'schedule':
            sampled_time = dm.sample_action(action, concept_name, resource)
            finish_time[task] = current_time + sampled_time
        elif action == 'start':
            sampled_time =  dm.sample_action(action, concept_name, resource)
            finish_time[task] += sampled_time
        elif action == 'suspend':
            sampled_time =  dm.sample_action(action, concept_name, resource)
            finish_time[task] += sampled_time
        else:
            current_time = max(current_time, finish_time[task])
        if action in ['schedule', 'start', 'suspend']:
            pass
            #print(action, concept_name, sampled_time)
        marking = semantics.execute(pn_task, net, marking)

    #print('total:', current_time)
    #print('---------')
    return start_time, current_time

kde_likelihood = lambda samples, ground_truth : gaussian_kde(samples)(ground_truth)


n = 1000
for case_name in event_log['case:concept:name'].unique():
    case_samples = [sample_end_time(case_name)[1] for i in range(n)]
    real_end_time = get_real_end_time(case_name)

    likelihood = kde_likelihood(case_samples, real_end_time)
    print(case_name, likelihood)
    
    plt.figure()
    plt.hist(case_samples, bins=100)
    plt.axvline(x=real_end_time, color='red')
    plt.show()
